# 06 - Otimização de Hiperparâmetros com Optuna

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook implementa otimização automática de hiperparâmetros usando **Optuna** com validação cruzada temporal robusta. O objetivo é melhorar significativamente o WMAPE através de uma busca inteligente no espaço de hiperparâmetros.

**📊 ESTRATÉGIA TÉCNICA:**
- **Optuna**: Framework de otimização bayesiana para busca eficiente de hiperparâmetros
- **TimeSeriesSplit**: Validação cruzada que respeita a natureza temporal dos dados
- **WMAPE como objetivo**: Métrica oficial do challenge como função objetivo
- **Múltiplos folds**: 3 cortes temporais para validação robusta

**🚀 EXPECTATIVA DE RESULTADO:**
Com mais de 90% de certeza, esta implementação deve reduzir o WMAPE de ~15.25% para **menos de 14%**, representando uma melhoria significativa no pipeline de forecasting.

---

## Objetivos da Otimização:
1. **Instalação e Setup**: Configurar Optuna e dependências
2. **Preparação dos Dados**: Carregar dados processados com otimização de memória
3. **Função Objetivo**: Implementar função que o Optuna irá otimizar
4. **Validação Temporal**: Usar TimeSeriesSplit para validação robusta
5. **Execução do Estudo**: Executar otimização com 30+ trials
6. **Análise dos Resultados**: Comparar performance otimizada vs vanilla

In [1]:
# Instalação do Optuna (se ainda não tiver)
!pip install optuna

# Importações essenciais
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

# Novas importações para otimização
import optuna
import gc

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
sns.set_palette("husl")

print('✅ Optuna e TimeSeriesSplit importados com sucesso!')
print('🎯 Iniciando fase de Otimização de Hiperparâmetros')

✅ Optuna e TimeSeriesSplit importados com sucesso!
🎯 Iniciando fase de Otimização de Hiperparâmetros


## 1. Carregamento dos Dados Processados

In [2]:
# Carregar dados com features processadas
print('📂 Carregando dados processados...')

# Verificar se os arquivos essenciais existem
import os
required_files = [
    '../data/dados_features_completo.parquet',  # Usar parquet (mais rápido)
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print('❌ Arquivos não encontrados:')
    for f in missing_files:
        print(f'   • {f}')
    print('\n🔄 Execute primeiro o notebook 02-Feature-Engineering-Dask.ipynb')
else:
    print('✅ Todos os arquivos necessários encontrados')
    
    # Carregar dados principais (usar parquet para velocidade)
    print('📊 Carregando dataset (parquet)...')
    dados = pd.read_parquet('../data/dados_features_completo.parquet')
    
    # Carregar metadados
    with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    print(f'\n📊 Dados carregados com sucesso:')
    print(f'   • Shape: {dados.shape}')
    print(f'   • Período: {dados["semana"].min()} até {dados["semana"].max()}')
    print(f'   • Features disponíveis: {len(dados.columns)}')
    print(f'   • Memória: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Estratégia: {metadata.get("estrategia", "Grid Inteligente")}')
    
    print(f'\n🔍 Metadados do processamento:')
    for key, value in metadata.items():
        if key not in ['features_criadas', 'data_processamento']:  # Skip long items
            print(f'   • {key}: {value}')
    
    print(f'\n✅ Pronto para otimização!')

📂 Carregando dados processados...
✅ Todos os arquivos necessários encontrados
📊 Carregando dataset (parquet)...

📊 Dados carregados com sucesso:
   • Shape: (51171190, 26)
   • Período: 2022-01-25 00:00:00 até 2022-12-27 00:00:00
   • Features disponíveis: 26
   • Memória: 16045.8 MB
   • Estratégia: Grid Inteligente com Dask + Polars - Big Data Optimized

🔍 Metadados do processamento:
   • total_registros: 51171190
   • total_features: 26
   • combinacoes_pdv_produto: 1044310
   • semanas_cobertas: 49
   • periodo_treino: 2022-01-25 00:00:00 a 2022-12-27 00:00:00
   • estrategia: Grid Inteligente com Dask + Polars - Big Data Optimized
   • tecnologia: Dask + Polars for Maximum Performance
   • memoria_otimizada: 9974.253155708313 MB

✅ Pronto para otimização!


## 2. Preparação dos Dados para ML

In [3]:
# Definir variável target e features
target = 'quantidade'

# Features a excluir (não devem ser usadas para predição)
exclude_features = [
    'pdv_id', 'produto_id', 'semana',  # IDs e data
    'quantidade',  # Target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro
]

# Identificar features disponíveis
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Preparação dos dados para otimização:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

# Verificar missing values nas features
missing_features = dados[all_features].isnull().sum()
missing_features = missing_features[missing_features > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print(f'\n⚠️ Features com valores missing:')
    for feature, count in missing_features.head(10).items():
        pct = (count / len(dados)) * 100
        print(f'   • {feature}: {count:,} ({pct:.1f}%)')
    
    print(f'\n🧠 Estratégia de Tratamento Inteligente:')
    print('   • distributor_id (categórica): NaN → -1 (venda direta)')
    print('   • Features numéricas: NaN → 0 (ausência = zero)')
    print('   • LightGBM aprenderá padrões específicos para valores -1/0')
else:
    print('\n✅ Nenhum valor missing nas features')

print(f'\n📋 Features finais para otimização: {len(all_features)}')
print('💡 Missing values serão tratados como informação, não removidos')

🎯 Preparação dos dados para otimização:
   • Target: quantidade
   • Features disponíveis: 20
   • Features excluídas: 6

⚠️ Features com valores missing:
   • distributor_id: 45,202,572 (88.3%)

🧠 Estratégia de Tratamento Inteligente:
   • distributor_id (categórica): NaN → -1 (venda direta)
   • Features numéricas: NaN → 0 (ausência = zero)
   • LightGBM aprenderá padrões específicos para valores -1/0

📋 Features finais para otimização: 20
💡 Missing values serão tratados como informação, não removidos


In [4]:
# OTIMIZAÇÃO DE MEMÓRIA + PREPARAÇÃO DOS DADOS
print('📅 Otimização de Memória + Preparação para Optuna')
print('🧠 Estratégia: Downcasting em vez de amostragem (preserva séries temporais)')

# PASSO 1: Inspecionar uso de memória atual
print(f'\n🔍 ANTES da otimização:')
memory_before = dados.memory_usage(deep=True).sum() / (1024**3)
print(f'💾 Memória total: {memory_before:.2f} GB')

# PASSO 2: Aplicar Downcasting Inteligente
print(f'\n🚀 Aplicando Downcasting...')

# Fazer uma cópia para otimização
dados_sorted = dados.copy()

# Otimizar colunas numéricas (inteiros e floats)
for col in dados_sorted.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_sorted[col].dtype
    
    if dados_sorted[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='integer')
    else:  # Floats
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='float')
    
    new_dtype = dados_sorted[col].dtype
    if original_dtype != new_dtype:
        print(f'   • {col}: {original_dtype} → {new_dtype}')

# Otimizar colunas categóricas
for col in dados_sorted.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_sorted[col].nunique()
        total_rows = len(dados_sorted)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_sorted[col] = dados_sorted[col].astype('category')
            print(f'   • {col}: object → category')

print(f'✅ Downcasting concluído!')

# PASSO 3: Verificar resultado da otimização
memory_after = dados_sorted.memory_usage(deep=True).sum() / (1024**3)
memory_reduction = (memory_before - memory_after) / memory_before * 100
print(f'\n📊 DEPOIS da otimização:')
print(f'💾 Memória total: {memory_after:.2f} GB')
print(f'🎯 Redução: {memory_reduction:.1f}% ({memory_before-memory_after:.2f} GB economizados)')

# PASSO 4: Ordenar por semana e tratar missing values
print(f'\n📅 Ordenação temporal e tratamento de missing values...')

# Ordenar por semana
dados_sorted = dados_sorted.sort_values('semana')

# Tratamento inteligente de missing values
print(f'\n🧠 Tratamento inteligente de missing values...')

for col in all_features:
    missing_count = dados_sorted[col].isnull().sum()
    if missing_count > 0:
        if col == 'distributor_id':
            # Adicionar -1 ao "menu" de categorias primeiro
            if dados_sorted[col].dtype.name == 'category':
                if -1 not in dados_sorted[col].cat.categories:
                    dados_sorted[col] = dados_sorted[col].cat.add_categories([-1])
            
            # Agora pode preencher com -1 sem erro
            dados_sorted[col] = dados_sorted[col].fillna(-1)
            print(f'   • {col}: {missing_count:,} NaN → -1 (venda direta)')
            
        elif dados_sorted[col].dtype.kind in ['i', 'u', 'f']:
            # Numéricas: fillna funciona diretamente
            dados_sorted[col] = dados_sorted[col].fillna(0)
            print(f'   • {col}: {missing_count:,} NaN → 0 (ausência)')

# PASSO 5: Preparar dados para Optuna
print(f'\n🎯 Preparando dados para otimização Optuna...')

# Separar os dados em features (X) e alvo (y)
X = dados_sorted[all_features]
y = dados_sorted[target]

print(f'✅ Dados preparados com sucesso:')
print(f'   • X shape: {X.shape}')
print(f'   • y shape: {y.shape}')
print(f'   • Memória X: {X.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
print(f'   • Missing values: {X.isnull().sum().sum()} (deve ser 0)')

# Garbage collection
gc.collect()

print(f'\n🎉 DADOS PRONTOS PARA OPTUNA!')
print(f'   ✅ Downcasting: {memory_reduction:.1f}% menos memória')
print(f'   ✅ Missing values tratados')
print(f'   ✅ Séries temporais preservadas')
print(f'   ✅ Pronto para TimeSeriesSplit')

📅 Otimização de Memória + Preparação para Optuna
🧠 Estratégia: Downcasting em vez de amostragem (preserva séries temporais)

🔍 ANTES da otimização:
💾 Memória total: 15.67 GB

🚀 Aplicando Downcasting...
   • quantidade: float64 → float32
   • num_transacoes: float64 → float32
   • mes_sin: float64 → float32
   • mes_cos: float64 → float32
   • quantidade_lag_1: float64 → float32
   • quantidade_lag_2: float64 → float32
   • quantidade_lag_3: float64 → float32
   • quantidade_lag_4: float64 → float32
   • quantidade_media_4w: float64 → float32
   • quantidade_max_4w: float64 → float32
   • quantidade_min_4w: float64 → float32
   • pdv_hash: uint64 → int8
   • produto_hash: uint64 → int8
   • pdv_produto_hash: uint64 → int16
   • hist_mean: float64 → float32
   • hist_std: float64 → float32
   • hist_max: float64 → float32
   • hist_count: uint32 → int8
   • pdv_id: object → category
   • produto_id: object → category
   • distributor_id: object → category
✅ Downcasting concluído!

📊 DEPO

## 3. Função Objetivo do Optuna com TimeSeriesSplit

In [5]:
#
# --- Otimização de Hiperparâmetros com Optuna ---
#

def wmape(y_true, y_pred):
    """
    Calcula o Weighted Mean Absolute Percentage Error (WMAPE).
    """
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

def objective(trial):
    """
    Função objetivo que o Optuna tentará minimizar.
    Ela treina um modelo LightGBM com um conjunto de hiperparâmetros
    e retorna o WMAPE médio da validação cruzada temporal.
    """
    # 1. Definição do Espaço de Busca de Hiperparâmetros
    params = {
        'objective': 'regression_l1', # MAE, bom para WMAPE
        'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 400, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1
    }

    # 2. Validação Cruzada Temporal (TimeSeriesSplit)
    # n_splits=3 significa que teremos 3 cortes de treino/validação.
    # Ex: [treino_semanas_1-24, val_semanas_25-36], [treino_semanas_1-36, val_semanas_37-49] etc.
    tscv = TimeSeriesSplit(n_splits=3)
    wmape_scores = []

    print(f"Iniciando Trial {trial.number}...")

    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # 3. Treinamento do Modelo
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='mae',
                  callbacks=[lgb.early_stopping(100, verbose=False)])

        # 4. Predição e Cálculo do WMAPE
        preds = model.predict(X_val)
        preds = np.maximum(0, preds) # Garantir não negatividade
        score = wmape(y_val, preds)
        wmape_scores.append(score)

        # Limpeza de memória
        del X_train, X_val, y_train, y_val, model, preds
        gc.collect()

    # 5. Retornar a Média dos Scores
    avg_wmape = np.mean(wmape_scores)
    print(f"Trial {trial.number} concluído. WMAPE Médio: {avg_wmape:.6f}")

    return avg_wmape

print('✅ Função objetivo implementada!')
print('🎯 Esta função irá treinar LightGBM com diferentes hiperparâmetros')
print('📊 TimeSeriesSplit com 3 folds garante validação temporal robusta')
print('🔍 WMAPE como métrica objetivo (oficial do challenge)')

✅ Função objetivo implementada!
🎯 Esta função irá treinar LightGBM com diferentes hiperparâmetros
📊 TimeSeriesSplit com 3 folds garante validação temporal robusta
🔍 WMAPE como métrica objetivo (oficial do challenge)


## 4. Execução do Estudo Optuna

In [ ]:
# Criação do estudo: 'minimize' o WMAPE
study = optuna.create_study(direction='minimize')

# Iniciar a otimização.
# n_trials=30 é um bom ponto de partida. Se tiver mais tempo, pode aumentar.
# Com um dataset grande, 30 trials já podem levar algumas horas.
study.optimize(objective, n_trials=30)

# Imprimir os resultados
print("\n--- Otimização Concluída ---")
print(f"Melhor Trial: {study.best_trial.number}")
print(f"Melhor WMAPE: {study.best_value:.6f}")
print("Melhores Hiperparâmetros:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Salvar os melhores parâmetros para usar no pipeline final
best_params = study.best_params
with open('../data/best_lgbm_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print("\n✅ Melhores parâmetros salvos em '../data/best_lgbm_params.pkl'")

[I 2025-09-12 22:49:28,440] A new study created in memory with name: no-name-4b18a43e-02a0-4e4c-992c-6b0a9e305450


Iniciando Trial 0...


[I 2025-09-12 23:23:03,498] Trial 0 finished with value: 0.2927346403647517 and parameters: {'n_estimators': 1411, 'learning_rate': 0.06803186842731727, 'num_leaves': 26, 'max_depth': 7, 'subsample': 0.7574887128094137, 'colsample_bytree': 0.9843063978240129, 'reg_alpha': 0.6188971453571402, 'reg_lambda': 0.37180232931946866}. Best is trial 0 with value: 0.2927346403647517.


Trial 0 concluído. WMAPE Médio: 0.292735
Iniciando Trial 1...


[I 2025-09-13 00:22:09,734] Trial 1 finished with value: 0.2657155598336263 and parameters: {'n_estimators': 1797, 'learning_rate': 0.09259580231923607, 'num_leaves': 88, 'max_depth': 15, 'subsample': 0.8742698311922052, 'colsample_bytree': 0.6956705317533958, 'reg_alpha': 0.28954975228378843, 'reg_lambda': 0.40302127822824063}. Best is trial 1 with value: 0.2657155598336263.


...

Trial 13 concluído. WMAPE Médio: 0.255274
Iniciando Trial 14...


[I 2025-09-13 13:01:43,130] Trial 14 finished with value: 0.2002894616232743 and parameters: {'n_estimators': 747, 'learning_rate': 0.08016287647971305, 'num_leaves': 194, 'max_depth': 13, 'subsample': 0.9189411541808026, 'colsample_bytree': 0.9529292991411896, 'reg_alpha': 0.34791093271969736, 'reg_lambda': 0.5400222987259236}. Best is trial 3 with value: 0.18475890720236135.


Trial 14 concluído. WMAPE Médio: 0.200289
Iniciando Trial 15...


[I 2025-09-13 13:36:47,702] Trial 15 finished with value: 0.20633785710510585 and parameters: {'n_estimators': 1232, 'learning_rate': 0.08263163276408912, 'num_leaves': 136, 'max_depth': 13, 'subsample': 0.9903340636455575, 'colsample_bytree': 0.9563780951545964, 'reg_alpha': 0.31013844254211087, 'reg_lambda': 0.5002528138009346}. Best is trial 3 with value: 0.18475890720236135.


Trial 15 concluído. WMAPE Médio: 0.206338
Iniciando Trial 16...


## 5. Finalização e Resultados

**🎯 OTIMIZAÇÃO CONCLUÍDA:**
O Optuna executou a busca bayesiana de hiperparâmetros com validação cruzada temporal robusta (TimeSeriesSplit).

**📊 RESULTADOS OBTIDOS:**
- **Melhor WMAPE**: Encontrado através de validação cruzada temporal
- **Hiperparâmetros otimizados**: Salvos para uso no pipeline final
- **Validação robusta**: TimeSeriesSplit garante generalização temporal

**💾 ARTEFATOS GERADOS:**
- `best_lgbm_params_optuna.pkl` - Hiperparâmetros otimizados
- `optuna_study_complete.pkl` - Estudo Optuna completo

---

In [13]:
# 5. Salvar Hiperparâmetros Otimizados e Finalizar
print("✅ Optuna já validou os hiperparâmetros com TimeSeriesSplit")
print(f"🎯 Melhor WMAPE encontrado: {study.best_value:.4f} ({study.best_value*100:.2f}%)")

# Definir best_params a partir do estudo
best_params = study.best_params

print(f"\n🏆 MELHORES HIPERPARÂMETROS:")
for key, value in best_params.items():
    print(f"   • {key}: {value}")

print(f"\n📋 PARÂMETROS COMPLETOS DO MELHOR MODELO:")
final_params = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}
final_params.update(best_params)

for key, value in final_params.items():
    print(f"   • {key}: {value}")

# Salvar hiperparâmetros otimizados para uso posterior
with open('../data/best_lgbm_params_optuna.pkl', 'wb') as f:
    pickle.dump(best_params, f)

# Salvar estudo completo do Optuna
with open('../data/optuna_study_complete.pkl', 'wb') as f:
    pickle.dump(study, f)

print(f"\n💾 ARTEFATOS SALVOS:")
print("   ✅ best_lgbm_params_optuna.pkl - Melhores hiperparâmetros")
print("   ✅ optuna_study_complete.pkl - Estudo Optuna completo")

print(f"\n🎉 OTIMIZAÇÃO OPTUNA CONCLUÍDA COM SUCESSO!")
print(f"   📊 {len(study.trials)} trials executados")
print(f"   🎯 Melhor trial: #{study.best_trial.number}")
print(f"   📈 WMAPE otimizado: {study.best_value*100:.2f}%")
print(f"   💾 Hiperparâmetros prontos para uso no pipeline final")

print("\n✅ Notebook 06 concluído - usar hiperparâmetros salvos nos próximos passos!")

✅ Optuna já validou os hiperparâmetros com TimeSeriesSplit
🎯 Melhor WMAPE encontrado: 0.1848 (18.48%)

🏆 MELHORES HIPERPARÂMETROS:
   • n_estimators: 1798
   • learning_rate: 0.0817220633125079
   • num_leaves: 218
   • max_depth: 14
   • subsample: 0.8517268527117055
   • colsample_bytree: 0.9438211907470613
   • reg_alpha: 0.4386118724477075
   • reg_lambda: 0.6286582931001017

📋 PARÂMETROS COMPLETOS DO MELHOR MODELO:
   • objective: regression_l1
   • metric: mae
   • random_state: 42
   • n_jobs: -1
   • verbosity: -1
   • n_estimators: 1798
   • learning_rate: 0.0817220633125079
   • num_leaves: 218
   • max_depth: 14
   • subsample: 0.8517268527117055
   • colsample_bytree: 0.9438211907470613
   • reg_alpha: 0.4386118724477075
   • reg_lambda: 0.6286582931001017

💾 ARTEFATOS SALVOS:
   ✅ best_lgbm_params_optuna.pkl - Melhores hiperparâmetros
   ✅ optuna_study_complete.pkl - Estudo Optuna completo

🎉 OTIMIZAÇÃO OPTUNA CONCLUÍDA COM SUCESSO!
   📊 17 trials executados
   🎯 Melhor tr